In [2]:
import pandas as pd
import numpy as np
from utils import lla_to_ecef

In [3]:
REF_LAT, REF_LON, REF_ALT = 42.294319, -83.223275, 148.30372

## GPS and IMU measurements

In [5]:
# read IMU data
imu_path = "../data/csv_data/imu.csv"
imu = pd.read_csv(imu_path)
imu['rosbagTimestamp'] = imu['secs']*int(1e9) + imu['nsecs']
imu = imu.drop(['header', 'seq', 'stamp', 'secs', 'nsecs'], axis=1)

imu.rename(columns={'frame_id':'has_imu', 'rosbagTimestamp':'time'}, inplace=True)
imu['has_imu'] = True

imu.rename(columns={f'{i}': f'imu_{i}' for i in ['ang_vel_x', 'ang_vel_y', 'ang_vel_z', 'lin_acc_x', 'lin_acc_y', 'lin_acc_z']}, inplace=True)

In [6]:
# read GPS data
gps_path = '../data/csv_data/gps.csv'
gps = pd.read_csv(gps_path)
gps.drop(['header', 'seq', 'stamp', 'secs', 'nsecs', 'status', 'status.1', 'service', 'position_covariance', 'position_covariance_type'], axis=1, inplace=True)
gps.rename(columns={'frame_id':'has_gps', 'rosbagTimestamp':'time'}, inplace=True)
gps['has_gps'] = True

# convert to ECEF
gps[['gps_pos_x', 'gps_pos_y', 'gps_pos_z']] = gps.apply(lambda row: lla_to_ecef(row.latitude, row.longitude, row.altitude), axis=1, result_type='expand')
gps.drop(['latitude', 'longitude', 'altitude'], axis=1, inplace=True)

# convert to local ECEF
gps

,time,has_gps,gps_pos_x,gps_pos_y,gps_pos_z
0,1508987519143971920,True,558059.529261,-4.692125e+06,4.269914e+06
1,1508987519147371053,True,558059.583527,-4.692125e+06,4.269914e+06
2,1508987519153815031,True,558059.637792,-4.692125e+06,4.269914e+06
3,1508987519157360076,True,558059.692041,-4.692125e+06,4.269914e+06
4,1508987519163784980,True,558059.746282,-4.692125e+06,4.269914e+06
...,...,...,...,...,...
155236,1508988347454502105,True,548368.985563,-4.698648e+06,4.264053e+06
155237,1508988347461013078,True,548368.940056,-4.698648e+06,4.264053e+06
155238,1508988347464492082,True,548368.894532,-4.698648e+06,4.264053e+06
155239,1508988347471018075,True,548368.849016,-4.698648e+06,4.264053e+06


In [4]:
# merge GPS and IMU
meas = pd.concat([gps, imu]).sort_values('time')
meas = meas.groupby('time').aggregate(max).reset_index()

# REMOVE TODO
meas['has_cam'] = False
meas

,time,has_gps,gps_pos_x,gps_pos_y,gps_pos_z,has_imu,imu_ang_vel_x,imu_ang_vel_y,imu_ang_vel_z,imu_lin_acc_x,imu_lin_acc_y,imu_lin_acc_z,has_cam
0,1508987519143971920,True,558059.529261,-4.692125e+06,4.269914e+06,True,0.002271,-0.004726,0.012533,-0.313470,0.121048,-0.281582,False
1,1508987519147371053,True,558059.583527,-4.692125e+06,4.269914e+06,True,0.004968,-0.005145,0.011527,-0.455330,0.147042,-0.183228,False
2,1508987519153815031,True,558059.637792,-4.692125e+06,4.269914e+06,True,0.008092,-0.003727,0.011873,-0.591017,0.290014,0.006007,False
3,1508987519157360076,True,558059.692041,-4.692125e+06,4.269914e+06,True,0.010363,0.000223,0.011455,-0.624363,0.199674,0.009783,False
4,1508987519163784980,True,558059.746282,-4.692125e+06,4.269914e+06,True,0.007671,0.000836,0.012655,-0.525673,0.066757,-0.181699,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
155234,1508988347454502105,True,548368.985563,-4.698648e+06,4.264053e+06,True,0.011993,0.015935,-0.005347,0.502658,-0.468592,0.672039,False
155235,1508988347461013078,True,548368.940056,-4.698648e+06,4.264053e+06,True,-0.000925,0.008811,-0.000713,0.568252,-0.228897,0.605711,False
155236,1508988347464492082,True,548368.894532,-4.698648e+06,4.264053e+06,True,-0.006713,-0.002176,0.003824,0.530415,0.303166,0.301102,False
155237,1508988347471018075,True,548368.849016,-4.698648e+06,4.264053e+06,True,0.006498,-0.001844,0.005197,0.316711,0.663943,0.297036,False


In [5]:
meas.to_csv('../data/measurements.csv')

## GPS ground truth

In [6]:
ground_truth = pd.DataFrame({'time':[0], 'x':[0], 'y':[0], 'z':[0]})
ground_truth.to_csv('../data/ground_truth.csv')